### Apache kafka Using PySpark Codeume

In [2]:
from pyspark.sql import SparkSession

In [11]:
# Initialize Spark Session
spark = (SparkSession.builder
    .appName("pysparkKafkaApp")
    .config("spark.streaming.stopGracefullyOnShutdown", True)
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0')
    .getOrCreate())

print("Spark Version:", spark.version)

Spark Version: 3.5.0


### Sending data from pyspark code to console of kafka

In [12]:
# Sample Data
data = spark.createDataFrame([
    (1, "Sending message to Kafka"),
    (2, "Sending message to Kafka again"),
    (3, "Message sent to Kafka")
], ["Id", "Message"])

print(data.show())

# Convert to JSON format (Kafka requires key-value messages)
data_json = data.selectExpr("CAST(Id AS STRING) AS key", "to_json(struct(*)) AS value")


+---+--------------------+
| Id|             Message|
+---+--------------------+
|  1|Sending message t...|
|  2|Sending message t...|
|  3|Message sent to K...|
+---+--------------------+

None
